# Local Memory Implementation for Matrix Multiplication

### Learning Objectives
- Able to describe how local memory is different than global memory and its advantages
- Able to implement local memory usage in SYCL kernel 
- Able to articulate advantages of using local memory.

## Shared Local Memory (SLM) Implementation

In a matrix multiplication algorithm, there is a high degree of reuse, so instead of loading values from global memory each time we can load the values into local memory and perform the computation.  This will reduce the latency of accessing the data values.  The difference between this implementation and the ND-range implementation is the reading is done from global memory in the case of ND-range each time and in this implementation both matrix A and B are loaded into local memory and then computed.  

When a work-group begins, the contents of its local memory are uninitialized, and local memory does not persist after a work-group finishes executing. Because of these properties, local memory may only be used for temporary storage while a work-group is executing.  For other devices though, such as many GPU devices, there are dedicated resources for local memory, and on these devices, communicating via local memory should perform better than communicating via global memory.

To declare local memory for use in an ND-range kernel, use a local accessor. Like other accessor objects, a local accessor is constructed within a command group handler, but unlike the accessor objects, a local accessor is not created from a buffer object.  From the following example, A_tile and B_tile are the local accessors we have defined that load the 16x16 items.  These are used to compute an intermediate result which does not use global memory for repeated access for computation.  

To simplify kernel development and accelerate communication between work-items in a work-group, SYCL defines a special local memory space specifically for communication between work-items in a work-group.

Also notice that we used a barrier that helps to synchronize all of the work-items in the work-group.  The performance is much better than the initial ND-range samples and slightly better than the ND-range sample utilizing local memory.

<img src=Assets/localmem.PNG>

## Matrix Multiplication with Local-memory

The following SYCL code shows a local-memory implementation of matrix multiplication: Inspect code; there are no modifications necessary:

1. Run the cell in the __Select Offload Device__ section to choose a target device to run the code on.
2. Inspect the following code cell and click __Run__ ▶ to save the code to a file.
3. Next, run the cell in the __Build and Run__ section to compile and execute the code.

#### Select Offload Device

In [ ]:
run accelerator.py

In [ ]:
%%writefile lab/mm_dpcpp_localmem.cpp
//==============================================================
// Matrix Multiplication: SYCL Local Accessor
//==============================================================
// Copyright © 2021 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================


#include <CL/sycl.hpp>

using namespace sycl;

void mm_kernel(queue &q, std::vector<float> &matrix_a, std::vector<float> &matrix_b, std::vector<float> &matrix_c, size_t N, size_t M) {
    std::cout << "Configuration         : MATRIX_SIZE= " << N << "x" << N << " | WORK_GROUP_SIZE= " << M << "x" << M << "\n";

    //# Create buffers for matrices
    buffer a(matrix_a);
    buffer b(matrix_b);
    buffer c(matrix_c);

    //# Submit command groups to execute on device
    auto e = q.submit([&](handler &h){
        //# Create accessors to copy buffers to the device
        auto A = a.get_access<access::mode::read>(h);
        auto B = b.get_access<access::mode::read>(h);
        auto C = c.get_access<access::mode::write>(h);

        //# Define size for ND-range and work-group size
        range<2> global_size(N,N);
        range<2> work_group_size(M,M);

        //# Create local accessors
        accessor<float, 2, access::mode::read_write, access::target::local> A_tile(range<2>(M, M), h);
        accessor<float, 2, access::mode::read_write, access::target::local> B_tile(range<2>(M, M), h);

        //# Parallel Compute Matrix Multiplication
        h.parallel_for(nd_range<2>{global_size, work_group_size}, [=](nd_item<2> item){
            const int i = item.get_global_id(0);
            const int j = item.get_global_id(1);
            const int x = item.get_local_id(0);
            const int y = item.get_local_id(1);

            float temp = 0.f;
            int k;
            for (int t = 0; t < N; t+=M) {
                A_tile[x][y] = A[i * N + (t + y)];
                B_tile[x][y] = B[(t + x) * N + j];
                item.barrier(access::fence_space::local_space);
                for (k = 0; k < M; k++) {
                    temp += A_tile[x][k] * B_tile[k][y];
                }
                item.barrier(access::fence_space::local_space);
            }
            C[i*N+j] = temp;
        });
    });
    c.get_access<access::mode::read>();
    
    //# print kernel compute duration from event profiling
    auto kernel_duration = (e.get_profiling_info<info::event_profiling::command_end>() - e.get_profiling_info<info::event_profiling::command_start>());
    std::cout << "Kernel Execution Time : " << kernel_duration / 1e+9 << " seconds\n";
}

#### Build and Run
Select the cell below and click __Run__ ▶ to compile and execute the code on selected device:

In [ ]:
! chmod 755 q; chmod 755 run_mm_localmem.sh; if [ -x "$(command -v qsub)" ]; then ./q run_mm_localmem.sh "{device.value}"; else ./run_mm_localmem.sh; fi

### Roofline Report

Execute the following line to display the roofline results 


In [ ]:
run display_data/mm_localmem_roofline.py

### VTune™ Profiler Summary

Execute the following line to display the VTune results.

In [ ]:
run display_data/mm_localmem_vtune.py

## Analysis

Comparing the execution times for ND-Range SYCL implementation and Shared Local Memory implementation for various matrix sizes, we can see thatshared local memory implementation performs better. The graph below shows execution times on various hardware for matrix sizes 1024x1024, 5120x5120 and 10240x10240.

<img src=Assets/ppp_ndrange_localmem_graph.PNG>

## Summary

Local memory continues the trend of improving performance across all GPU platforms.  In addition, local memory has thus far provided the best performance as compared to all other optimizations. In this module we learned how local memory is different from global memory and how to implement local memory.  Choosing a CPU for an offload device also exhibits the same behavior and we are able to use the identical code across all platforms.  

In the next module we will compare the relative performance across all platforms.

- Notices -- Intel technologies may require enabled hardware, software or service activation.
No product or component can be absolutely secure. Your costs and results may vary. © Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. *Other names and brands may be claimed as the property of others.